## This jupyter notebook compares the sage version of macaulay.py from autoguess to the without sage version

# Below is the sage version

In [8]:
#!/usr/local/bin/sage -python3
'''
Created on Feb 14, 2021

@author: Hosein Hadipour
@contact: hsn.hadipour@gmail.com

Given a set of Boolean polynomials and a positive integer D, as well as a monomial ordering, 
this class performs two main tasks:
    1- Constructing the Macaulay matrix of degree D
    2- Computing the reduced row echelon form of the derived Macaulay matrix
'''

from sage.all import *
from sage.rings.polynomial.multi_polynomial_sequence import PolynomialSequence
import time
import sys
from datetime import datetime
from argparse import ArgumentParser, RawTextHelpFormatter

class Macaulay:
    """
    Given a set of Boolean polynomials and a positive integer D, as well as a monomial ordering, 
    this class performs two main tasks:
    1- Constructing the Macaulay matrix of degree D
    2- Computing the reduced row echelon form of the derived Macaulay matrix
    """

    count = 0

    def __init__(self, inputfile, outputfile, D=2, term_ordering='deglex'):
        Macaulay.count += 1
        self.inputfile = inputfile
        self.outputfile = outputfile
        self.D = D
        self.term_ordering = term_ordering
        self.algebrize_input_polynomials()

    @parallel
    def get_vars(self, symbolic_equations):
        return list(symbolic_equations.variables())
    
    @parallel
    def ring_evaluator(self, eq):
        return self.PolyRing(eq)

    def algebrize_input_polynomials(self):
        """
        This function converts the given polynomials in string format to a list of boolean polynomials of type PolynomialSequence
        """
        t0 = time.time()
        try:
            with open(self.inputfile, 'r') as equations_file:
                string_polynomials = equations_file.read().splitlines()
        except IOError:
            print('%s is not accessible!' % self.inputfile)
            sys.exit()
        symbolic_polynomials = list(map(symbolic_expression, string_polynomials))
        symbolic_variables = list(set(flatten(list(map(self.get_vars, symbolic_polynomials)))))
        self.PolyRing = BooleanPolynomialRing(len(symbolic_variables), names=symbolic_variables, order=self.term_ordering)
        self.polynomial_sequence = PolynomialSequence(list(map(self.ring_evaluator, symbolic_polynomials)))
        print(f"algebrize_input_polynomials done in {time.time() - t0:.4f} seconds")
        
    def build_macaulay_polynomials(self):
        """
        It constrcuts the Macaulay polynomials with degree at most D corresponding to the given boolean polynomaisl sequence
        """
        t0 = time.time()
        self.macaulay_polynomials = []
        nvars = self.PolyRing.n_variables()
        degree_spectrum = sorted(set([f.degree() for f in self.polynomial_sequence]))
        print('Number of algebraic equations: %d' % len(self.polynomial_sequence))
        print('Number of algebraic variables: %d' % nvars)
        print('Number of algebraic monomials: %d' % self.polynomial_sequence.nmonomials())
        print('Spectrum of degrees: %s' % degree_spectrum)
        multiplied_monomials = dict()
        for d in degree_spectrum:
            print(f"d: {d} ", end = " ")
            if d < self.D:
                maximal_exponents = IntegerVectors(self.D - d, nvars, max_part = 1)
                leading_terms = [self.PolyRing({tuple(exponent):1}) for exponent in maximal_exponents]
                monomials = [[mono for mono in list(lead_term.lead_divisors())] for lead_term in leading_terms]
                monomials = list(set(flatten(monomials)))
                multiplied_monomials[d] = monomials
            else:
                multiplied_monomials[d] = [1]
            print(f"mul: {multiplied_monomials[d]}")
        for f in self.polynomial_sequence:        
            self.macaulay_polynomials.extend([m*f for m in multiplied_monomials[f.degree()]])
        self.macaulay_polynomials = PolynomialSequence(self.macaulay_polynomials)   
        print(f"build_macaulay_polynomials done in {time.time() - t0:.4f} seconds")
    
    def build_macaulay_matrix(self):
        """
        This function generates the Macaulay matrix
        """
        t0 = time.time()
        if self.polynomial_sequence.maximal_degree() > 1:
            minimum_degree = min([f.degree() for f in self.polynomial_sequence])
            if self.D < minimum_degree:
                self.D = minimum_degree
            self.build_macaulay_polynomials()
            self.macaulay_matrix, self.macaulay_vars = self.macaulay_polynomials.coefficient_matrix()
        else:
            self.macaulay_matrix, self.macaulay_vars = self.polynomial_sequence.coefficient_matrix()
        print(self.macaulay_matrix)
        print('Macaulay matrix was generated in %s' % str.lower(self.macaulay_matrix.parent().__repr__()))
        print(f"build_macaulay_matrix done in {time.time() - t0:.4f} seconds")
    
    def gaussian_elimination(self):
        """
        This function applies compute the row redecued echelon form of the derived Macaulay matrix
        """
        t0 = time.time()
        print('Gaussian elimination was started - %s' % datetime.now())
        start_time = time.time()
        self.macaulay_matrix.echelonize()
        elapsed_time = time.time() - start_time
        self.dependent_vars = [self.macaulay_vars[i][0] for i in self.macaulay_matrix.pivots() \
            if not self.macaulay_vars[i][0] in [0, 1]]
        self.dependent_vars = list(map(str, self.dependent_vars))
        self.free_vars = [self.macaulay_vars[i][0] for i in self.macaulay_matrix.nonpivots() \
            if not self.macaulay_vars[i][0] in [0, 1]]
        self.free_vars = list(map(str, self.free_vars))
        # print(self.free_vars)
        # print(self.dependent_vars)
        print('#Dependent variables: %d' % len(self.dependent_vars))
        print('#Free variables: %d' % len(self.free_vars))
        print('Gaussian elimination was finished after %0.2f seconds' % elapsed_time)
        print(f"gaussian_elimination done in {time.time() - t0:.4f} seconds")

    # def write_result(self):
    #     """
    #     This function aims to write the derived polynomials into the output file
    #     """

    #     nrows = self.macaulay_matrix.nrows()
    #     print('Writing the results into the %s - %s' % (self.outputfile, datetime.now()))
    #     starting_time = time.time()
    #     with open(self.outputfile, 'w') as outputfile:
    #         for i in range(nrows):
    #             output = " + ".join([str(self.macaulay_vars[j, 0]) for j in self.macaulay_matrix.nonzero_positions_in_row(i)])
    #             if output != '':
    #                 output += "\n"
    #                 outputfile.write(output)
    #     elapsed_time = time.time() - starting_time
    #     print('Result was written into %s after %0.02f seconds' % (self.outputfile, elapsed_time))

# def loadparameters(args):
#     """
#     Get parameters from the argument list and inputfile.
#     """

#     # Load default values
#     params = {"inputfile": "example2.txt",
#               "outputfile": "macaulay_basis.txt",
#               "D" : 2,
#               "term_ordering": 'deglex'}

#     # Override parameters if they are set on commandline
#     if args.inputfile:
#         params["inputfile"] = args.inputfile[0]

#     if args.outputfile:
#         params["outputfile"] = args.outputfile[0]
    
#     if args.D:
#         params["D"] = args.D[0]
    
#     if args.term_ordering:
#         params["term_ordering"] = args.term_ordering[0]

#     return params

# def main():
#     """
#     Parse the arguments and start the request functionality with the provided
#     parameters.
#     """

#     parser = ArgumentParser(description="This tool computes the Macaulay matrix with degree D,"
#                                         " given a system of boolean polynomials and"
#                                         " a positive integer D",
#                             formatter_class=RawTextHelpFormatter)
#     parser.add_argument('-i', '--inputfile', nargs=1, help="Use an input file in plaintext format to"
#                                                      " read the relations from.")
#     parser.add_argument('-o', '--outputfile', nargs=1, help="Use an output file to"
#                         " write the output into it.")
#     parser.add_argument('-D', '--D', nargs=1, type=int,
#                         help="A positive integer as the degree of Macaulay matrix.")
#     parser.add_argument('-t', '--term_ordering', nargs=1, type=str,
#                         help="A term ordering such as deglex or degrevlex.")

#     # Parse command line arguments and construct parameter list.
#     args = parser.parse_args()
#     params = loadparameters(args)
#     macaulay = Macaulay(inputfile=params['inputfile'], outputfile=params['outputfile'], \
#         D=params['D'], term_ordering=params['term_ordering'])
#     macaulay.build_macaulay_matrix()
#     macaulay.gaussian_elimination()
#     macaulay.write_result()

# if __name__ == '__main__':
#     main()

inputfile = "eqn.txt"
outputfile = "output.txt"
D = 3

macaulay = Macaulay(inputfile=inputfile, outputfile = outputfile, D=D)
# macaulay.build_macaulay_polynomials()
macaulay.build_macaulay_matrix()
# macaulay.gaussian_elimination()
print(f"Total Macaulay polynomials: {len(macaulay.macaulay_polynomials)}")


algebrize_input_polynomials done in 0.0075 seconds
Number of algebraic equations: 1
Number of algebraic variables: 3
Number of algebraic monomials: 3
Spectrum of degrees: [2]
d: 2  mul: [1, X1, X3, x1]
build_macaulay_polynomials done in 0.0004 seconds
[0 0 0 1 1 0 0 1]
[0 1 0 1 0 1 0 0]
[0 0 1 1 0 0 1 0]
[1 0 0 0 0 0 0 0]
Macaulay matrix was generated in full matrixspace of 4 by 8 sparse matrices over finite field of size 2
build_macaulay_matrix done in 0.0014 seconds
Total Macaulay polynomials: 4


# Below is the without sage version

In [10]:
import time
import sys
from datetime import datetime
from argparse import ArgumentParser, RawTextHelpFormatter
from itertools import combinations


# from sympy import sympify, expand, Pow, GF, Poly, total_degree
# import numpy as np
# sys.path.insert(0, "/home/tools/autoguess/venv/lib/python3.11/site-packages")
import galois
from gf2poly import parse, GF2Poly


def poly_degree(poly: GF2Poly) -> int:
    """
    Return the total degree of a GF2Poly:
      max number of variables in any monomial.
    """
    # handle the zero polynomial
    if not poly.monomials:
        return 0
    return max(len(mono) for mono in poly.monomials)

class Macaulay:
    
    count = 0
    
    def __init__(self, inputfile, outputfile, D=2, term_ordering='deglex'):
        Macaulay.count += 1
        self.inputfile = inputfile
        self.outputfile = outputfile
        self.D = D
        self.term_ordering = term_ordering
        self.algebraize_input_polynomials()
        
    def algebraize_input_polynomials(self):
        t0 = time.time()
        try:
            with open(self.inputfile, 'r') as eqf:
                # Skip empty lines and comments
                lines = [L.strip() for L in eqf
                         if L.strip() and not L.strip().startswith('#')]
        except IOError:
            print(f'{self.inputfile} is not accessible!')
            sys.exit(1)
        
        # 1) Parse each algebraic relation into GF2Poly
        self.polynomials = [parse(line) for line in lines]  # list of GF2Poly

        # 2) Collect all variable names from monomials
        vars_set = set()
        for poly in self.polynomials:
            for mono in poly.monomials:    # mono is a frozenset of var-names
                vars_set.update(mono)
        self.variable_names = sorted(vars_set)  # e.g. ['x0','x1','x2',…]
        
        elapsed = time.time() - t0
        # print(f'Parsed {len(self.polynomials)} GF2Polys with '
        #       f'{len(self.variable_names)} variables in {elapsed:.3f}s')
        print(f"algebrize_input_polynomials done in {time.time() - t0:.4f} seconds")
    
    def build_macaulay_polynomials(self):
        """
        Constructs the Macaulay polynomials (all in GF2Poly form)
        of total degree ≤ D, given the input GF2Polys in self.polynomials.
        """
        t0 = time.time()

        # 1) Prepare
        self.macaulay_polynomials = []
        nvars = len(self.variable_names)

        # 2) Compute degrees of each input GF2Poly via our poly_degree helper
        degrees = [poly_degree(f) for f in self.polynomials]
        degree_spectrum = sorted(set(degrees))

        # print(f"  • Degree spectrum = {degree_spectrum}  "
        #     f"(computed in {time.time()-t0:.4f}s)")
        # print(f"  • # equations = {len(self.polynomials)}, "
        #     f"# variables = {nvars}")
        unique_monos = set()
        for poly in self.polynomials:
            unique_monos |= poly.monomials   # union in all monomials
    
        print('Number of algebraic equations: %d' % len(self.polynomials))
        print('Number of algebraic variables: %d' % nvars)
        print('Number of algebraic monomials: %d' % len(unique_monos))
        print('Spectrum of degrees: %s' % degree_spectrum)


        # 3) Pre-compute the monomial multipliers for each degree d
        #    so that d + deg(multiplier) ≤ D
        multipliers = {}
        for d in degree_spectrum:
            r = self.D - d
            print(f"d: {d}")
            if r < 0:
                # if an equation already exceeds D, just multiply by 1
                multipliers[d] = [GF2Poly([frozenset()])]
                continue

            mons = []
            # generate all monomials of degree k = 0..r
            for k in range(r + 1):
                for idxs in combinations(range(nvars), k):
                    # build the frozenset of names for this monomial
                    mono_vars = frozenset(self.variable_names[i] for i in idxs)
                    mons.append(GF2Poly([mono_vars]))
            multipliers[d] = mons
            print(f"mul: {multipliers[d]}")
        # total_mults = sum(len(v) for v in multipliers.values())
        
        
        # print(f"  • Generated {total_mults} multipliers in {time.time()-t0:.4f}s")

        # 4) Multiply each input poly by its allowed multipliers,
        #    collecting only the non-zero products
        count = 0
        for f_poly, d in zip(self.polynomials, degrees):
            for m in multipliers[d]:
                prod = m * f_poly          # GF2Poly __mul__
                if prod:                   # skip the zero polynomial
                    self.macaulay_polynomials.append(prod)
                    count += 1

        # print(f"  • Built {count} Macaulay polynomials "
        #     f"in {time.time()-t0:.4f}s")
        print(f"build_macaulay_polynomials done in {time.time() - t0:.4f} seconds")
        
    def build_macaulay_matrix(self):
        """
        Generate the Macaulay matrix over GF(2) from GF2Poly-based macaulay_polynomials.
        """
        t0 = time.time()

        # 1) Decide if we need Macaulay lifting (nonlinear case)
        degrees = [poly_degree(f) for f in self.polynomials]
        # print(f"  • Max input degree = {max(degrees)} (computed in {time.time()-t0:.4f}s)")

        if max(degrees) > 1:
            min_deg = min(degrees)
            if self.D < min_deg:
                self.D = min_deg
            self.build_macaulay_polynomials()
        else:
            # print("  • All linear ⇒ skipping lift")
            # just treat inputs as your Macaulay polys
            self.macaulay_polynomials = list(self.polynomials)

        # td = time.time()

        # 2) Collect & sort all monomials (as exponent‐tuples) in graded-lex order
        var_names = self.variable_names
        exps = set()
        for poly in self.macaulay_polynomials:
            for mono in poly.monomials:
                # mono is frozenset of var‐names → build an exp tuple
                exps.add(tuple(1 if v in mono else 0 for v in var_names))

        # graded-lex: by total degree, then lex tuple, descending
        self.mons = sorted(exps, key=lambda e: (sum(e), e), reverse=True)
        col_index = {exp: i for i, exp in enumerate(self.mons)}

        # print(f"  • Collected {len(self.mons)} monomials in {time.time()-td:.4f}s")

        # 3) Build the 0/1 matrix
        n_rows = len(self.macaulay_polynomials)
        n_cols = len(self.mons)
        A = np.zeros((n_rows, n_cols), dtype=np.uint8)

        for i, poly in enumerate(self.macaulay_polynomials):
            for mono in poly.monomials:
                exp = tuple(1 if v in mono else 0 for v in var_names)
                A[i, col_index[exp]] = 1

        self.macaulay_matrix = A
        print(A)

        # ti = time.time()
        # print(f"  • Built matrix {n_rows}×{n_cols} in {ti-t0:.4f}s")
        print(f"Macaulay matrix was generated in full matrixspace of {n_rows} by {n_cols} sparse matrices over finite field of size 2")
        print(f"build_macaulay_matrix done in {time.time() - t0:.4f} seconds")

    def gaussian_elimination(self):
        """
        Perform RREF on GF(2) matrix using galois, and manually compute pivots.
        """
        t0 = time.time()
        GF2 = galois.GF(2)
        A = GF2(self.macaulay_matrix)
        
        self.R = A.row_reduce()
        
        # convert to numpy to inspect bit patterns
        R_np = np.array(self.R, dtype=np.uint8)

        pivots = [int(np.nonzero(R_np[i])[0][0])
                for i in range(R_np.shape[0])
                if R_np[i].any()]
        
        def is_constant(mono):
            return all(e == 0 for e in mono)
        

        # Exclude constant monomial from free/dependent count
        self.dependent = [m for j, m in enumerate(self.mons) if j in pivots and not is_constant(m)]
        self.free = [m for j, m in enumerate(self.mons) if j not in pivots and not is_constant(m)]

                # self.dependent = [self.mons[j] for j in pivots]
                # self.free = [self.mons[j] for j in range(self.macaulay_matrix.shape[1]) if j not in pivots]

        elapsed = time.time() - t0
        print(f"#Dependent variables: {len(self.dependent)}")
        print(f"#Free variables: {len(self.free)}")
        print(f"Gaussian elimination was finished after {elapsed:.4f} seconds")
    
    def write_result(self):
        """
        Writes the reduced equations (rows of self.R) to self.outputfile
        using variable names in self.variable_names and monomials in self.mons.
        Also prints each equation to the console.
        """
        t0 = time.time()
        try:
            with open(self.outputfile, 'w') as outf:
                # R should be your reduced-row-echelon form (NumPy array or list of lists)
                rows = np.array(self.R, dtype=np.uint8)

                for row in rows:
                    terms = []
                    for j, bit in enumerate(row):
                        if bit:
                            mono = self.mons[j]             # e.g. (0,1,0,1,0,...)
                            if sum(mono) == 0:
                                # constant term
                                terms.append('1')
                            else:
                                # select the variable names for exponent=1
                                var_list = [
                                    self.variable_names[i]
                                    for i, e in enumerate(mono) if e
                                ]
                                terms.append('*'.join(var_list))

                    if terms:
                        equation = ' + '.join(terms)
                        outf.write(equation + '\n')
                        print(equation)

        except IOError:
            print(f"Error: cannot write to {self.outputfile}")
            sys.exit(1)

        elapsed = time.time() - t0
        print(f"Result written to {self.outputfile} in {elapsed:.4f} seconds")
    
inputfile = "eqn.txt"
outputfile = "chacha_output.txt"
D = 3

macaulay = Macaulay(inputfile=inputfile, outputfile=outputfile, D=D)
# macaulay.build_macaulay_polynomials()
macaulay.build_macaulay_matrix()
macaulay.gaussian_elimination()
print(f"Total Macaulay polynomials: {len(macaulay.macaulay_polynomials)}")

algebrize_input_polynomials done in 0.0004 seconds
Number of algebraic equations: 7
Number of algebraic variables: 4
Number of algebraic monomials: 11
Spectrum of degrees: [2]
d: 2
mul: [1, X1, X2, X3, X4]
build_macaulay_polynomials done in 0.0001 seconds
[[0 0 0 0 0 1 0 0 1 0 1 0 1 1 0]
 [0 1 0 0 0 0 1 0 0 0 1 0 0 0 0]
 [1 0 0 0 1 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 1 0 0 1 0 0]
 [0 0 1 0 0 0 1 0 1 1 0 0 0 1 0]
 [0 0 0 0 0 0 1 1 0 1 1 1 0 1 0]
 [1 0 1 0 1 0 0 0 0 0 1 0 0 0 0]
 [0 1 0 1 1 0 0 1 1 0 0 1 0 0 0]
 [0 0 1 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 1 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 1 1 0 1 0 1]
 [0 1 1 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 1 1 0 0 1 0 0 0]
 [0 0 0 1 0 1 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 0 0 1 0]
 [0 0 0 0 1 1 0 1 0 0 0 0 1 1 1]
 [1 0 0 0 1 0 1 0 0 0 1 0 0 0 0]
 [1 0 0 0 1 0 0 0 1 0 0 1 0 0 0]
 [1 0 0 0 0 1 0 1 0 1 0 0 0 0 0]
 [0 1 1 1 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 1 1 0 0 0 0 1 0 0]
 [1 0 0 0 1 1 1 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 0 0 0 